In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [3]:
X,y=make_classification(n_samples=10000, n_features=10, n_informative=3)

In [4]:
X

array([[ 1.92412535, -1.02542035,  1.24782868, ..., -2.13945889,
        -1.3798111 ,  1.64853769],
       [ 1.32881327, -1.4775985 ,  2.12556933, ..., -0.59183817,
         0.51098167, -0.12204928],
       [-0.87679522, -0.13001661, -2.55839405, ...,  0.67199283,
        -1.58686434,  1.22230976],
       ...,
       [ 0.46341664,  0.88768893,  2.69162906, ...,  0.60429237,
         0.24371112,  0.25200266],
       [-0.33960317,  0.36556045, -0.65003408, ..., -1.49279296,
        -1.24708574,  1.13281061],
       [-0.64840205, -0.2167324 , -0.21279713, ...,  0.8834271 ,
         2.36341792, -2.46224453]])

In [5]:
X_train , X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))
     

Decision Tree accuracy 0.9746666666666667


bagging

In [7]:
bg_clf=BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=500, max_samples=0.5, bootstrap=True, random_state=42)

In [8]:
bg_clf.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [9]:
y_pred=bg_clf.predict(X_test)
print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.9813333333333333


bagging with SVM

In [10]:
bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)
     

bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Bagging using SVM",accuracy_score(y_test,y_pred))

Bagging using SVM 0.9766666666666667


pasting

pasting ans bagging me bootstrap=true and false ka hi difference hai sirf


In [11]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,
    n_jobs=-1
)
     

bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))
     

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    7.1s remaining:    7.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    7.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Pasting classifier 0.982


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


random subspaces

In [12]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,   # ye btata hai ki for each base model kitni no of rows jayengi 
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)
     

bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

Random Subspaces classifier 0.9816666666666667


random patches 

In [13]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)
     

bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))
     


Random Patches classifier 0.9796666666666667


OOB score

In [14]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)
     

bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [15]:
bag.oob_score_

0.9788571428571429

In [16]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.9813333333333333


**Bagging Tips**
Bagging generally gives better results than Pasting
Good results come around the 25% to 50% row sampling mark
Random patches and subspaces should be used while dealing with high dimensional data
To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV